In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize

from stardist import random_label_cmap, _draw_polygons
from stardist.models import StarDist2D

np.random.seed(6)
lbl_cmap = random_label_cmap()

Using TensorFlow backend.


In [2]:
#---------#4idapi#--------#
from skimage import io
from skimage.filters import threshold_otsu
import Paths

folder = '/links/groups/treutlein/DATA/imaging/PW/4i/plate6/AlignedOrganoids/MAX_Time00000_Point0000_Point00{ii}_ChannelSCF_SD'



membrane = io.imread(folder + "/cycles/cycle1_aligned.tif")[...,1]

dapi = io.imread(folder + "/cycles/cycle1_aligned.tif")[...,2]

def subtract_membrane(dapi, membrane):
    dapi = scale_image(dapi)
    membrane = scale_image(membrane)
    tmp = dapi - membrane
    tmp[tmp < 0] = 0
    thresh = threshold_otsu(tmp)
    tmp[tmp < thresh] = 0
    return scale_image(tmp)

def scale_image(image, percentile = 5):
        return np.interp(image, (np.percentile(image,percentile), np.percentile(image,100 - percentile)), (0, +65535))

subtracted = subtract_membrane(dapi, membrane)
#subtracted = subtracted[500:1500,500:1500]

In [4]:
model_paper = StarDist2D(None, name='2D_dsb2018', basedir=Paths.accessory_data_path + 'trained_CNN_stardist_paper/')

NameError: name 'Paths' is not defined

In [ ]:
img = normalize(subtracted, 1,99.8, axis=axis_norm)
labels, details = model_paper.predict_instances(img)

fig, ax = plt.subplots(1,3,figsize=(20,20))
ax[0].imshow(dapi)
ax[1].imshow(subtracted)
plt.axis('off')
ax[2].imshow(img if img.ndim==2 else img[...,0], clim=(0,1), cmap='gray')
ax[2].imshow(labels, cmap=lbl_cmap, alpha=0.5)
plt.axis('off');

In [ ]:
plt.figure(figsize = (30,30))
plt.imshow(img if img.ndim==2 else img[...,0], clim=(0,1), cmap='gray')
plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
plt.axis('off')
plt.show